## Import libraries

- Follow instructions on this link to sign up for Google Earth Engine - https://geemap.org/installation/

In [1]:
import ee
# ee.Authenticate()
ee.Initialize()

In [2]:
import sys
import os
import geemap
# import geemap.colormaps as cm

In [3]:
print(f"""System: {sys.version} 
geemap: {geemap.__version__}
ee: {ee.__version__} 
        """)     

System: 3.12.0 | packaged by conda-forge | (main, Oct  3 2023, 08:36:57) [Clang 15.0.7 ] 
geemap: 0.30.4
ee: 0.1.388 
        


In [4]:
# example AOI to search Landsat images
polygon = ee.Geometry.Polygon([
    [[-92.625046, 38.351888],
  [-92.625046, 38.416342],
  [-92.52943, 38.416342],
  [-92.52943, 38.351888],
  [-92.625046, 38.351888]]
  ])

## Get Landsat-8 images
[Info](https://developers.google.com/earth-engine/guides/landsat)

In [5]:
# first we create some functions to mask cloudy pixels and scaling of bands
def maskL8sr(image):
    """Function to mask clouds and shodows based on the pixel_qa band of Landsat 8 SR collection.
    Args: {ee.Image} image Input Landsat 8 SR image
    Return: {ee.Image} Cloudmasked Landsat 8 image"""
    
   # cloud and shadow mask for a given image using the Collection 2 QA bitmask codes.
    qa_mask = image.select('QA_PIXEL').bitwiseAnd(int('11111', 2)).eq(0)
    return image.updateMask(qa_mask).copyProperties(image, ["system:time_start"])


# Applies scaling factors to Landsat 8 images.
def apply_scale_factors(image):
  optical_bands = image.select('SR_B.').multiply(0.0000275).add(-0.2)
  thermal_bands = image.select('ST_B.*').multiply(0.00341802).add(149.0)
  return image.addBands(optical_bands, None, True).addBands(
      thermal_bands, None, True
  )

In [6]:
# get image collection
ls8 = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2') \
    .filterDate('2023-05-01', '2023-06-01') \
    .filterMetadata('CLOUD_COVER', 'less_than', 40) \
    .filterBounds(polygon) \
    .map(apply_scale_factors) \
    .map(maskL8sr) \
    .select(['SR_B2', 'SR_B3', 'SR_B4', 'SR_B5']) #select band names to include
print(f'Total number of images = {ls8.size().getInfo()}')

Total number of images = 3


In [7]:
# Visualize using geemap
Map = geemap.Map(center=(40, -100), zoom=4, basemap="HYBRID")
Map.centerObject(polygon)
visualization = {
    'bands': ['SR_B4_median', 'SR_B3_median', 'SR_B2_median'],
    'min': 0.0,
    'max': 0.3,
}

Map.addLayer(ls8.reduce(ee.Reducer.median()).clip(polygon), visualization, 'Median RGB')

Map

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(ch…

## Export images in the collection to google drive

In [8]:
# Export the image collection to Google Drive.
geemap.ee_export_image_collection_to_drive(ls8,
                                           folder='landsat_images', #Name of the google drive folder
                                           scale=30, region=polygon)

# # Can also use the following to save locally for small exports
# geemap.ee_export_image_collection(ls8, '{path to the local folder}',
#                                     scale= 30, region=polygon)

Total number of images: 3

